In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import shutil

In [2]:
response = requests.get('http://www.imfdb.org/wiki/Category:Gun')
soup = BeautifulSoup(response.text, 'html.parser')
base_link = 'http://www.imfdb.org'

In [3]:
category_pages = []
tables = soup.find_all('table')
category_table = tables[5]
category_links = category_table.find_all('a', href = True)
for i in category_links: 
    category_pages.append(f"{base_link}{i['href']}")

In [4]:
gun_links = []
category_pages = {'Assault_Rifle':'http://www.imfdb.org/wiki/Category:Assault_Rifle', 
                   'Pistol': 'http://www.imfdb.org/wiki/Category:Pistol'}
#                 'Revolver':'http://www.imfdb.org/wiki/Category:Revolver', 
#                   'Shotgun': 'http://www.imfdb.org/wiki/Category:Shotgun', 
#                     'Rifle': 'http://www.imfdb.org/wiki/Category:Rifle'}
for pos, categ in tqdm(enumerate(category_pages.keys()), desc = 'Getting Gun Links'): 
    page = category_pages[categ]
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'html.parser')
    gun_table = soup.find_all('table')
    gun_l= gun_table[-1].find_all('a', href = True)
    for i in gun_l:
        gun_links.append((f"{base_link}{i['href']}", categ))

Getting Gun Links: 2it [00:00,  2.93it/s]


In [5]:
#gun_links

In [6]:
def download_image(image): 
    response = requests.get(image[0], stream = True)
    num = image[2]
    name = image[1]
    file_name = f'{name}_{num}'
    file = open(f'Images/{name}/StockImages/{file_name}.jpg', 'wb')
    response.raw.decode_content = True
    shutil.copyfileobj(response.raw, file)
    del response
#download_image(gun_image_info[0],0)

In [7]:
media_links = []
media_names = []
gun_image_info = []
for pos, (gl, name) in enumerate(tqdm(gun_links, desc = 'Getting Links to Movies/TV/VideoGames')): 
  
    response = requests.get(gl)
    soup = BeautifulSoup(response.text, 'html.parser')
    gun_image = soup.find('img', class_ = 'thumbimage')
    try: 
        gun_image = (f"{base_link}{gun_image['src']}", name, pos)
        #gun_image_info.append(gun_info)
        download_image(gun_image)
    except: 
        continue

    #gets links within each table
    tables = soup.find_all('table')
    if len(tables) == 0: 
        continue
    for table in tables: 
        col = [i.text.strip() for i in table.find_all('th')]
        if len(col) == 0 or len(col) < 2:
            continue
        if col[1] != 'Actor':
            continue
        else: 
            links = table.find_all('a', href = True)
            for i in links: 
                media_links.append(f"{base_link}{i['href']}")
                media_names.append(name)
#     table = tables[0]
#     links = table.find_all('a', href = True)
#     for i in links: 
#         media_links.append(f"{base_link}{i['href']}")
#         media_names.append(name)
    
    
#     for table in tables: 
#         links = table.find_all('a', href = True)
#         for i in links: 
#             media_links.append(f"{base_link}{i['href']}")
#             media_names.append(name)
    
    
    

Getting Links to Movies/TV/VideoGames: 100%|██████████| 304/304 [01:51<00:00,  2.72it/s]


In [8]:
pd.DataFrame({'URL': media_links, 'Name': media_names}).to_csv('LinksToMedia.csv', index = False)        

In [9]:
# for pic in (gun_image_info): 
#     download_image(pic)

In [10]:
# import pandas as pd
# pd.DataFrame({'URL': media_links}).to_csv('LinksToMedia.csv', index = False)